In [1]:
import pandas as pd 
import math
import os
from dotenv import load_dotenv
from alpha_vantage.timeseries import TimeSeries

In [2]:
#retorno diário e acumulado
#melhor e pior desempenho
#calculo de alpha e beta
#análise de drawdown maximo
#retorno anualizado
#volatilidade anualizada
#indice sharpe anualizado
#visualização - gráfico de linha dos retornos anualizados
#grafico de barra de retornos anualizados
#grafico de barras da volatilidade anualizada
#gráficos de barras do indice de sharpe
#gráfico de disperção (Benchmark no eixo X e retornos diários no eixo Y, adicionar linha de regressão para visualizar relação usada no Beta)

In [ ]:
#Carregando dados de Invesco, JP Morgan Chase e Lilly
load_dotenv()
api_key = os.getenv('ALPHA_VANTAGE_KEY')
ts = TimeSeries(key=api_key, output_format='pandas')
ivz, meta_data = ts.get_daily(symbol='IVZ', outputsize='full')
lly, meta_data = ts.get_daily(symbol='LLY', outputsize='full')
jpm, meta_data = ts.get_daily(symbol='JPM', outputsize='full')

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2025-07-18,17.895,20.045,17.710,19.92,35673235.0
2025-07-17,16.850,17.295,16.845,17.28,5059031.0
2025-07-16,16.870,16.960,16.490,16.88,5831685.0
2025-07-15,17.370,17.430,16.730,16.74,6692396.0
2025-07-14,17.390,17.445,17.245,17.34,4294841.0
...,...,...,...,...,...
1999-11-05,44.750,44.750,44.500,44.50,2680.0
1999-11-04,44.500,44.560,44.250,44.56,25400.0
1999-11-03,44.500,44.880,44.000,44.75,17200.0


In [ ]:
#Tratamento de dados
c_ivz = pd.DataFrame(ivz['4. close'])
c_ivz.index = pd.to_datetime(c_ivz.index)
c_ivz = c_ivz.sort_index(ascending=True)

c_lly = pd.DataFrame(lly['4. close'])
c_lly.index = pd.to_datetime(c_lly.index)
c_lly = c_lly.sort_index(ascending=True)

c_jpm = pd.DataFrame(jpm['4. close'])
c_jpm.index = pd.to_datetime(c_jpm.index)
c_jpm = c_jpm.sort_index(ascending=True)


In [ ]:
#Retonos acumulados
daily_return_ivz = c_ivz.pct_change()
daily_return_lly = c_lly.pct_change()
daily_return_jpm = c_jpm.pct_change()

cummulated_daily_return_ivz = (1 + daily_return_ivz).cumprod() - 1
cummulated_daily_return_lly = (1 + daily_return_lly).cumprod() - 1
cummulated_daily_return_jpm = (1 + daily_return_jpm).cumprod() - 1


print(cummulated_daily_return_ivz.tail())
print(cummulated_daily_return_jpm.tail())
print(cummulated_daily_return_lly.tail())


            4. close
date                
2025-07-14 -0.621480
2025-07-15 -0.634578
2025-07-16 -0.631522
2025-07-17 -0.622790
2025-07-18 -0.565160
            4. close
date                
2025-07-14  2.455002
2025-07-15  2.429272
2025-07-16  2.420536
2025-07-17  2.469363
2025-07-18  2.485759
             4. close
date                 
2025-07-14  10.626764
2025-07-15  10.225455
2025-07-16  10.488000
2025-07-17  10.076364
2025-07-18  10.224873


In [ ]:
#Melhor e pior retorno
best_return_stock = pd.Series({
    'lly':cummulated_daily_return_lly['4. close'].iloc[-1],
    'jpm':cummulated_daily_return_jpm['4. close'].iloc[-1],
    'ivz':cummulated_daily_return_ivz['4. close'].iloc[-1],
})


print('Melhor desempenho acumulado:\n', best_return_stock.sort_values(ascending=False))
print('Pior desempenho acumulado:\n', best_return_stock.sort_values(ascending=True))

Melhor desempenho acumulado:
 lly    10.224873
jpm     2.485759
ivz    -0.565160
dtype: float64
Pior desempenho acumulado:
 ivz    -0.565160
jpm     2.485759
lly    10.224873
dtype: float64


In [ ]:
#Puxando dados do benchmark
spy, meta_data = ts.get_daily(symbol='spy', outputsize='full')

c_spy = pd.DataFrame(spy['4. close'])
c_spy.index = pd.to_datetime(c_spy.index)
c_spy = c_spy.sort_index(ascending=True)

daily_return_spy = c_spy.pct_change()
    

In [31]:
#Tratamento de dados para calcular o beta
returns_spy = daily_return_spy['4. close']
returns_ivz = daily_return_ivz['4. close']
returns_jpm = daily_return_jpm['4. close']
returns_lly = daily_return_lly['4. close']

data_for_beta = pd.DataFrame({
    'spy_returns': returns_spy,
    'ivz_returns': returns_ivz,
    'jpm_returns': returns_jpm,
    'lly_returns': returns_lly
}).dropna()

matrix_covariance = data_for_beta.cov()

In [32]:
#Calculo do beta
covariance_spy_ivz = matrix_covariance.loc['spy_returns', 'ivz_returns']
covariance_spy_jpm = matrix_covariance.loc['spy_returns', 'jpm_returns']
covariance_spy_lly = matrix_covariance.loc['spy_returns', 'lly_returns']

variance_spy = data_for_beta['spy_returns'].var()

beta_ivz = covariance_spy_ivz/variance_spy
beta_jpm = covariance_spy_jpm/variance_spy
beta_lly = covariance_spy_lly/variance_spy

beta_ivz, beta_jpm, beta_lly

(1.6085957968818796, 1.3665895948807205, 0.6766710033362777)

In [46]:
#Calculo do alpha
r_f = 0.05
r_f_daily = (1 + r_f) ** (1/252) - 1
mean_benchmark_returns = returns_spy.mean()
mean_ivz_returns = returns_ivz.mean()
mean_jpm_returns = returns_jpm.mean()
mean_lly_returns = returns_lly.mean()

expected_return_ivz = r_f_daily + beta_ivz * (mean_benchmark_returns - r_f_daily)
expected_return_jpm = r_f_daily + beta_jpm * (mean_benchmark_returns - r_f_daily)
expected_return_lly = r_f_daily + beta_lly * (mean_benchmark_returns - r_f_daily)

daily_alpha_ivz = mean_ivz_returns - expected_return_ivz
daily_alpha_jpm = mean_jpm_returns - expected_return_jpm
daily_alpha_lly = mean_lly_returns - expected_return_lly

annualized_alpha_ivz = (1 + daily_alpha_ivz) ** (252/1) -1
annualized_alpha_jpm = (1 + daily_alpha_jpm) ** (252/1) - 1
annualized_alpha_lly = (1 + daily_alpha_lly) ** (252/1) -1

print('alpha ivz', annualized_alpha_ivz*100)
print('alpha jpm', annualized_alpha_jpm*100)
print('alpha lly', annualized_alpha_lly*100)


alpha ivz -1.774030418081196
alpha jpm 3.099192292828934
alpha lly 6.592777391920612
